In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
%pip install -Uq "unstructured[all-docs]" pillow lxml pillow
%pip install -Uq chromadb tiktoken
%pip install -Uq langchain langchain-community langchain-openai langchain-groq
%pip install -Uq python_dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Extracting the data 

from unstructured.partition.pdf import partition_pdf

output_path = ''
file_path = '/Users/felipesilverio/Documents/GitHub/LangChainTest/test2.pdf'

chunks = partition_pdf(
    filename= file_path,
    infer_table_structure=True, #extracting table
    strategy = 'hi_res', #mandatory to infer table

    extract_image_block_types=['Image'], #add 'Table' to list to extract image of tables
    # image_output_dir_path = output_path, #if None, images and tables will be saved as base64

    extract_image_block_to_payload=True, #if true, extract base64 for API usage

    chunking_strategy='by_title', #or basic
    max_characters=10000, #default is 500
    combine_text_under_n_chars=2000, #default is 0
    new_after_n_chars=6000, #default is 0
)

chunks

/Users/felipesilverio/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [13]:
import pandas as pd 
import numpy as np

chunks[20].metadata.orig_elements

In [14]:
elements = chunks[20].metadata.orig_elements

chunk_cap = [el for el in elements if 'Table' in str(type(el))]
chunk_cap[0].to_dict()


{'type': 'Table',
 'element_id': '904cdf8b-9d93-4a97-8f0a-d27e7ebb9dba',
 'text': 'Interest receivable on loans to group undertakings 2023 2022 € € 3,419,870 3,333,666 3,419,870 3,333,666',
 'metadata': {'detection_class_prob': 0.37135007977485657,
  'coordinates': {'points': ((np.float64(183.71788024902344),
     np.float64(1031.6480712890625)),
    (np.float64(183.71788024902344), np.float64(1265.5126953125)),
    (np.float64(1528.5313720703125), np.float64(1265.5126953125)),
    (np.float64(1528.5313720703125), np.float64(1031.6480712890625))),
   'system': 'PixelSpace',
   'layout_width': 1653,
   'layout_height': 2339},
  'last_modified': '2025-07-21T17:03:57',
  'text_as_html': '<table><thead><tr><th></th><th>2023</th><th>2022</th></tr><tr><th></th><th>€</th><th>€</th></tr></thead><tbody><tr><td rowspan="2">Interest receivable on loans to group undertakings</td><td>3,419,870</td><td>3,333,666</td></tr><tr><td>3,419,870</td><td>3,333,666</td></tr></tbody></table>',
  'filetype': '

In [3]:
tables, texts, images= [], [], []

for chunk in chunks:
    if "CompositeElement" in str(type(chunk)):
        chunk_els = chunk.metadata.orig_elements
        for el in chunk_els:
            if "Table" in str(type(el)):
                tables.append(el)
            elif "Image" in str(type(el)):
                images.append(el)
            else:
                texts.append(el)

In [39]:
chunks[0].metadata.orig_elements

## Creating model

In [3]:
pip list | grep -E "langchain|chromadb|tiktoken"


chromadb                                 1.0.15
langchain                                0.3.27
langchain-community                      0.3.27
langchain-core                           0.3.72
langchain-groq                           0.3.6
langchain-openai                         0.3.28
langchain-text-splitters                 0.3.9
tiktoken                                 0.9.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [24]:
prompt_text = """
You are an especialist in corporate finance tasked with summarizing the text, tables and images.
Give a concise summary of the table, text or image.
The tables will be received in format html. Transform this format in order to interpret the table.

The summary must take special attention to financial-related numbers and statistics, such as monthly or yearly comparisons, debt/loan information, and other subjects related.
The summary must contain the numerical information of debt, loan, revenue, deficit, and other related topics.

Response only with the summary, no additional comment. 
Do not start your message by saying "Here is a summary" or anything like that. 
Just give the summart as it is.

Table or text chunk: {element}
"""

prompt = ChatPromptTemplate.from_template(prompt_text)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5,)
# model = ChatGroq(temperature=0.5, model='llama-3.1-8b-instant')
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [25]:
# text_input = [{'input':text} for text in texts]

text_summaries = summarize_chain.batch(texts, {'max_concurrency':3})
text_summaries 


["TOUR PARTNER GROUP LIMITED reported a total revenue of $15 million for the fiscal year, with a year-over-year growth of 10%. The company's total debt stands at $5 million, reflecting a debt-to-equity ratio of 0.25. Operating expenses were recorded at $10 million, resulting in a net profit of $5 million. The company has a cash reserve of $2 million and a current ratio of 1.5, indicating good short-term financial health. Additionally, there was a deficit of $1 million in the previous quarter, which has been addressed in the current financial period.",
 "The annual report includes key financial statements highlighting the company's performance over the fiscal year. It presents total revenue of $2.5 million, a 15% increase from the previous year. Operating expenses totaled $1.8 million, resulting in an operating income of $700,000. The report indicates a net profit of $500,000 after accounting for interest expenses and taxes. \n\nTotal liabilities are reported at $1.2 million, with long-

In [26]:
# table_input = [{'input': html} for html in tables]
tables_html = [table.metadata.text_as_html for table in tables]

table_summaries = summarize_chain.batch(tables, {'max_concurrency':3})
table_summaries

['The provided text includes information about the directors of a company, P J Maine and A Graves, along with the registered number (09810143) and the registered office located at Hygeia Building, Harrow, Middlesex. It also mentions Xeinadin Audit Limited as the independent auditors and statutory auditors, based in London. There are no financial-related numbers or statistics, such as debt, loan information, revenue, or deficits, present in this text.',
 "The document includes a strategic report, directors' report, independent auditors' report, and financial statements. Key financial sections include the statement of comprehensive income, statement of financial position, and statement of changes in equity. Specific numerical details regarding revenue, expenses, debt, and equity changes are likely provided in the financial statements and notes, which should outline key financial metrics such as total revenue, net income, total assets, total liabilities, and equity changes over the report

In [2]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

vectorstore = Chroma(collection_name='multi_modal_rag', embedding_function=OpenAIEmbeddings())

#Storage layor
store = InMemoryStore()
id_key = 'doc_id'

#retriever

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

ModuleNotFoundError: No module named 'langchain._api'

In [20]:
retriever.docstore

In [29]:
#Loading values

doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]}) for i, summary in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
tables_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=summary, metadata={id_key: tables_ids[i]}) for i, summary in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(tables_ids, tables)))

In [31]:
# import fitz
# import matplotlib.patches as patches
# import matplotlib.pyplot as plt
# from PIL import Image


# def plot_pdf_with_boxes(pdf_page, segments):
#     pix = pdf_page.get_pixmap()
#     pil_image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

#     fig, ax = plt.subplots(1, figsize=(10, 10))
#     ax.imshow(pil_image)
#     categories = set()
#     category_to_color = {
#         'Title':'orchid',
#         'Image':'forestgreen',
#         'Table':'tomato'
#     }

#     for segment in segments:
#         points = segment['coordinates']['points']
#         layout_width = segment['coordinates']['layout_width']
#         layouth_height = segment['coordinates']['layout_height']
#         scaled_points =[
#             (x * pix.width / layout_width, y * pix.height / layouth_height)
#             for x, y in points
#         ]
#         box_color = category_to_color.get(segment['category'], 'deepskyblue')
#         categories.add(segment['category'])
#         rect = patches.Polygon(
#             scaled_points, linewidth=1, edgecolor=box_color, facecolor='none'
#         )
#         ax.add_patch(rect)

#     legend_handles = [patches.Patch(color='deepskyblue', label='Text')]
#     for category in ['Title','Image','Table']:
#         if category in categories:
#             legend_handles.append(
#                 patches.Patch(color=category_to_color[category], label=category)
#             )
#     ax.axis('off')
#     ax.legend(handles=legend_handles, loc='upper right')
#     plt.tight_layout()
#     plt.show()

# def render_page(doc_list, page_number, print_text=True):
#     pdf_page = fitz.open(file_path).load_page(page_number - 1)
#     page_docs = [
#         doc for doc in doc_list if doc.metadata.get('page_number') == page_number
#     ]

#     segments = [doc.metadata for doc in page_docs]
#     plot_pdf_with_boxes(pdf_page, segments)
#     if print_text:
#         for doc in page_docs:
#             print(f'{doc.page_content}\n')





chunks = retriever.invoke(
    # 'Which companies are mentioned in this file?'
    'Provide a summary of the Facility A and Facility B in terms of all financial metrics linked to two these facilities'
)

for doc in chunks:
    print(str(doc) + "\n\n" + "-" * 80)

2023 2022 2023 2022 GBP GBP EUR EUR Facility A 2,375,000 2,375,000 2,739,149 2,681,066 Facility B 8,800,000 8,800,000 10,149,269 9,934,056

--------------------------------------------------------------------------------
Facility A and B accrue interest at a variable rate equivalent to SONIA plus 3.5% and 4% respectively, and is payable quarterly.

--------------------------------------------------------------------------------


## RAG Pipeline

In [32]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from base64 import b64decode

def parse_docs(docs):
    # Split base64 images and texts
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return{'images':b64, 'texts':text}


def build_prompt(kwargs):
    
    docs_by_type = kwargs['context']
    user_question = kwargs['question']

    context_text = ""
    if len(docs_by_type['texts']) > 0:
        for text_element in docs_by_type['texts']:
            context_text += text_element.text

    #construct prompt with context
    prompt_template = f""" 
    Answuer the question based only on the following context, which can include text, tables and image.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{'type':'text','text':prompt_template}]

    if len(docs_by_type['images']) > 0:
        for image in docs_by_type['images']:
            prompt_content.append(
                {
                    'type': 'image_url',
                    'image_url': {'url': f'data:image/jpeg;base64,{image}'},
                }
            )

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )


chain = (
    {
        'context': retriever | RunnableLambda(parse_docs),
        'question': RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | ChatOpenAI(model='gpt-4o-mini')
    | StrOutputParser()
)

chain_with_sources = {
    'context': retriever | RunnableLambda(parse_docs),
    'question': RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | ChatOpenAI(model='gpt-4o')
        | StrOutputParser()
    )
)

In [35]:
response = retriever.invoke(
    # 'Which companies are mentioned in this file?'
    'What are the two loan facilities: Facility A and Facility B? Provide me a summary with detailed information'
)

for doc in response:
    print(str(doc) + "\n\n" + "-" * 80)

Term loan facilities A & B

--------------------------------------------------------------------------------
Facility A and B accrue interest at a variable rate equivalent to SONIA plus 3.5% and 4% respectively, and is payable quarterly.

--------------------------------------------------------------------------------


In [36]:
response = chain.invoke(
    # 'Which companies are mentioned in this file?'
    'What are the two loan facilities: Facility A and Facility B? '
)

print(response)

The two loan facilities are Facility A and Facility B. Facility A accrues interest at a variable rate equivalent to SONIA plus 3.5%, while Facility B accrues interest at a variable rate equivalent to SONIA plus 4%. Both facilities have interest that is payable quarterly.


In [40]:
response = chain_with_sources.invoke(
    # 'Which companies are mentioned in this file?'
    'What are the two loan facilities: Facility A and Facility B? Provide me a detailed summary'
)

print(response)
print(response['response'])

{'context': {'images': [], 'texts': [<unstructured.documents.elements.Title object at 0x31c995040>, <unstructured.documents.elements.NarrativeText object at 0x31c995880>]}, 'question': 'What are the two loan facilities: Facility A and Facility B? Provide me a detailed summary', 'response': "Facility A and Facility B are term loan facilities that accrue interest at a variable rate. The interest rate for each facility is determined by adding a fixed percentage to the Sterling Overnight Index Average (SONIA), which is a benchmark interest rate for sterling-denominated transactions.\n\n- **Facility A**: This loan facility accrues interest at a rate equivalent to SONIA plus 3.5%. This means the interest rate is variable and adjusts with fluctuations in the SONIA rate, plus an additional 3.5%. The interest payments for Facility A are due on a quarterly basis.\n\n- **Facility B**: This loan facility has a similar structure but accrues interest at SONIA plus 4%. Like Facility A, the interest r

# USING FINANCE TEAM PROMPT

In [57]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from base64 import b64decode
from prompts import system_finance_prompt

def parse_docs(docs):
    # Split base64 images and texts
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return{'images':b64, 'texts':text}


def build_prompt_two(kwargs) -> ChatPromptTemplate:
    """
    Construct a ChatPromptTemplate that always begins with the system prompt,
    then includes context (text + images) and the user question.
    """

    context = kwargs['context']
    question = kwargs['question']

    # Concatenate all text fragments
    context_text = "".join([t.text for t in context.get('texts', [])])

    # Build the messages list: SystemMessage -> HumanMessage
    messages = [
        SystemMessage(content=system_finance_prompt),
        HumanMessage(content=f"Context: {context_text}\nQuestion: {question}")
    ]

    # Include images if present
    for b64 in context.get('images', []):
        messages.append(
            HumanMessage(content={
                'type': 'image_url',
                'image_url': {'url': f'data:image/jpeg;base64,{b64}'},
            })
        )

    # Create and return a prompt template from these messages
    return ChatPromptTemplate.from_messages(messages)


chain = (
    {
        'context': retriever | RunnableLambda(parse_docs),
        'question': RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt_two)
    | ChatOpenAI(model='o3')
    | StrOutputParser()
)

chain_with_sources = {
    'context': retriever | RunnableLambda(parse_docs),
    'question': RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt_two)
        | ChatOpenAI(model='o3')
        | StrOutputParser()
    )
)

In [135]:
my_prompt = 'Make the company profile of Tour Partner Group Limited. Even if it is not available: revenue split by geography or segment, full cash-flow statement or EBITDA reconciliation, or other problems.'
response = chain_with_sources.invoke(my_prompt)

print(response)

{'context': {'images': [], 'texts': [<unstructured.documents.elements.NarrativeText object at 0x31ba7b0d0>]}, 'question': 'Make the company profile of Tour Partner Group Limited. Even if it is not available: revenue split by geography or segment, full cash-flow statement or EBITDA reconciliation,', 'response': 'Tour Partner Group Limited  \nAll data, unless otherwise stated, are taken from the company’s statutory Annual Report & Financial Statements for the year ended 31 Dec-23, filed at UK Companies House on 27-Jun-24.  \nSource references for every data-point are provided in-line (― “AR23, p.xx”).  \n\n=====================================================================\n1. Introduction Table (Company Snapshot)\n=====================================================================\nPrimary industry              : Travel services  \nIncorporation year            : 2015  –  10 Jul-15 (Companies House)  \nHeadquarters                  : London, United Kingdom  \nEmployees (exact)      

In [136]:
print(response['response'])

Tour Partner Group Limited  
All data, unless otherwise stated, are taken from the company’s statutory Annual Report & Financial Statements for the year ended 31 Dec-23, filed at UK Companies House on 27-Jun-24.  
Source references for every data-point are provided in-line (― “AR23, p.xx”).  

1. Introduction Table (Company Snapshot)
Primary industry              : Travel services  
Incorporation year            : 2015  –  10 Jul-15 (Companies House)  
Headquarters                  : London, United Kingdom  
Employees (exact)             : 334 (AR23, p.35)  

Operational KPIs (FY23)  
• Passengers handled          : 117,642 pax (AR23, p.6)  
• Tours operated              : 4,821 programmes (AR23, p.6)  
• Source markets served       : 61 origin countries (AR23, p.7)  

2. Business Overview (Bullets Only)
• Tour Partner Group Limited is a B2B inbound tour operator focusing on the UK, Ireland, and Scandinavian destinations, delivering group and FIT programmes on behalf of international t

In [69]:
print(response['response'])

Tour Partner Group Limited – Restructuring Screening Profile  
(All information is extracted verbatim from publicly-available filings at UK Companies House unless stated otherwise.  Latest statutory accounts filed: year-ended 31-Dec-22, submitted 29-Sep-23; Companies House document ID: 130247110)  

--------------------------------------------------------------------------------------------------------------------
1.  Company Snapshot  

Primary industry             :  Travel & Tourism  
Incorporation year           :  Oct-15  
Headquarters                 :  London, United Kingdom  
Employees (average FY22)     :  199                                (Source: FY22 Directors’ Report, p.14)  

Operational KPIs (FY22, same source unless noted)  
• Passengers handled         :  126,000  
• Source markets             :  41 countries  
• Destination Management Co. :  5 in-house DMC brands (Hotels & More, Irish Welcome Tours, Authentic Vacations, Trans Nordic Tours, Experience Scotland)  

---

In [138]:
from PyPDF2 import PdfReader

reader = PdfReader('/Users/felipesilverio/Documents/GitHub/LangChainTest/output/company_profile_unicode.pdf')
reader

all_text = []
for page_num, page in enumerate(reader.pages, start=1):
    try:
        text = page.extract_text() or ''
    except Exception:
        text = ''
    all_text.append(text)

teste = "\n".join(all_text)
teste

'Tour Partner Group Limited – Restructuring Screening Profile\n(All information is extracted verbatim from publicly-available filings at UK Companies House unless\nstated otherwise. Latest statutory accounts filed: year-ended 31-Dec-22, submitted 29-Sep-23;\nCompanies House document ID: 130247110)\n--------------------------------------------------------------------------------------------------------------------\n1. Company Snapshot\nPrimary industry : Travel & Tourism\nIncorporation year : Oct-15\nHeadquarters : London, United Kingdom\nEmployees (average FY22) : 199 (Source: FY22 Directors’ Report, p.14)\nOperational KPIs (FY22, same source unless noted)\n\x7f Passengers handled : 126,000\n\x7f Source markets : 41 countries\n\x7f Destination Management Co. : 5 in-house DMC brands (Hotels & More, Irish Welcome Tours,\nAuthentic Vacations, Trans Nordic Tours, Experience Scotland)\n----------------------------------------------------------------------------------------------------------

In [75]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from pptx import Presentation
from pptx.util import Inches

def generate_pdf_from_text(text: str, output_path: str):
    """
    Creates a PDF file from plain text.
    
    Parameters:
    - text: the string content to write
    - output_path: full file path where PDF will be saved
    """
    c = canvas.Canvas(output_path, pagesize=letter)
    width, height = letter
    text_obj = c.beginText(Inches(1), height - Inches(1))
    for line in text.split('\n'):
        text_obj.textLine(line)
        # Start a new page if we reach the bottom margin
        if text_obj.getY() < Inches(1):
            c.drawText(text_obj)
            c.showPage()
            text_obj = c.beginText(Inches(1), height - Inches(1))
    c.drawText(text_obj)
    c.save()

def generate_pptx_from_text(text: str, output_path: str):
    """
    Creates a PowerPoint (.pptx) presentation from plain text.
    
    Each paragraph (separated by blank lines) becomes a new slide.
    
    Parameters:
    - text: the string content to convert
    - output_path: full file path where PPTX will be saved
    """
    prs = Presentation()
    slide_layout = prs.slide_layouts[1]  # Title + Content layout
    sections = text.split('\n\n')
    
    for section in sections:
        lines = section.strip().split('\n')
        title = lines[0][:60]  # Use the first line as slide title (truncate if necessary)
        body_lines = lines[1:]
        
        slide = prs.slides.add_slide(slide_layout)
        slide.shapes.title.text = title
        text_frame = slide.shapes.placeholders[1].text_frame
        for line in body_lines:
            p = text_frame.add_paragraph()
            p.text = line
            p.level = 0
    
    prs.save(output_path)

# pip install fpdf
from fpdf import FPDF

def text_to_pdf(text: str, output_path: str):
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    
    # multi_cell handles wrapping and newlines for you
    for line in text.split('\n'):
        pdf.multi_cell(0, 8, line)
    
    pdf.output(output_path)
    
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet

def text_to_pdf_unicode(text: str, output_path: str):
    """
    Creates a PDF file from plain text with full Unicode support
    using ReportLab's Platypus framework.
    
    Parameters:
    - text: the string content to write into the PDF.
    - output_path: full file path where the PDF will be saved.
    """
    # 1. Prepare the document
    doc = SimpleDocTemplate(output_path, pagesize=letter)
    styles = getSampleStyleSheet()
    body_style = styles['BodyText']
    
    # 2. Build a "story" of flowable objects
    story = []
    for line in text.split('\n'):
        # Paragraph handles Unicode (e.g. “–”, “é”, emojis, etc.) natively
        story.append(Paragraph(line or ' ', body_style))
        # Small spacer between lines
        story.append(Spacer(1, 4))
    
    # 3. Generate the PDF
    doc.build(story)

# Example usage:
# Assuming you have a LangChain chain and have already run:
# result = chain_with_sources.invoke("Make the company profile of Tour Partner Group Limited")
# profile_text = result["response"]

profile_text = response["response"]  # fetched from your chain output
generate_pdf_from_text(profile_text, "/Users/felipesilverio/Documents/GitHub/LangChainTest/output/company_profile.pdf")
generate_pptx_from_text(profile_text, "/Users/felipesilverio/Documents/GitHub/LangChainTest/output/company_profile.pptx")
# text_to_pdf(profile_text, "/Users/felipesilverio/Documents/GitHub/LangChainTest/output/company_profile2.pdf")
text_to_pdf_unicode(profile_text, "/Users/felipesilverio/Documents/GitHub/LangChainTest/output/company_profile_unicode.pdf")

# After running, you’ll find:
# - /mnt/data/company_profile.pdf
# - /mnt/data/company_profile.pptx


# READING POWERPOINT (working)

In [85]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
from typing import Any, Dict, List, Optional

def extract_slide_elements(file_path: str):
    """
    Read and classify shapes from a PowerPoint file.

    Returns a mapping of slide indices to a list of element dictionaries,
    each containing a 'type' and relevant metadata or content.
    """
    try:
        presentation = Presentation(file_path)
    except Exception as e:
        raise IOError(f"Unable to open file {file_path}: {e}")

    slides_data: Dict[int, List[Dict[str, Any]]] = {}
    for idx, slide in enumerate(presentation.slides, start=1):
        elements: List[Dict[str, Any]] = []
        for shape in slide.shapes:
            # Images
            if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                img = shape.image
                elements.append({
                    'type': 'image',
                    'name': getattr(img, 'filename', None),
                    'width': shape.width,
                    'height': shape.height
                })

            # Tables
            elif hasattr(shape, 'table') and shape.has_table:
                table = shape.table
                data = [[cell.text for cell in row.cells] for row in table.rows]
                elements.append({'type': 'table', 'data': data})

            # Charts
            elif shape.shape_type == MSO_SHAPE_TYPE.CHART:
                chart = shape.chart
                elements.append({
                    'type': 'chart',
                    'chart_type': chart.chart_type,
                    'series_count': len(chart.series)
                })

            # Text boxes, placeholders, etc.
            elif shape.has_text_frame:
                text = "\n".join(
                    ''.join(run.text for run in paragraph.runs)
                    for paragraph in shape.text_frame.paragraphs
                )
                elements.append({'type': 'text', 'text': text})

            # Other shapes (auto-shapes, lines, etc.)
            else:
                elements.append({
                    'type': 'other',
                    'shape_type': shape.shape_type
                })

        slides_data[idx] = elements

    return slides_data

slides = extract_slide_elements('/Users/felipesilverio/Documents/GitHub/LangChainTest/One-Pager Structure_Reference for Felipe.pptx')

In [79]:
slides[1]

[{'type': 'text', 'text': '{COMPANY NAME}'},
 {'type': 'text', 'text': '{MONTH} -\xa0{YEAR}'},
 {'type': 'other', 'shape_type': <MSO_SHAPE_TYPE.PLACEHOLDER: 14>},
 {'type': 'image', 'name': 'image.png', 'width': 6046330, 'height': 6868458}]

In [83]:
slides[2]

[{'type': 'text', 'text': '{COMPANY_NAME} - Company Overview'},
 {'type': 'text', 'text': ''},
 {'type': 'text', 'text': 'Key Recent Developments'},
 {'type': 'text', 'text': 'Financial Highlight (FY22-24)'},
 {'type': 'text', 'text': 'Business Overview'},
 {'type': 'text',
  'text': 'Sources: Company financial statements, Company website, Debtwire, CIQ, Bloomberg, News\nNotes: 1) Advisors to the company; 2) The values for LTM 1Q25 is not included in the financials table as the update does not provide detailed cash flow data; 3) Excluding net working capital and taxes paid'},
 {'type': 'text', 'text': 'Key Stakeholders'},
 {'type': 'text', 'text': 'Revenue Split (FY24)'},
 {'type': 'text', 'text': 'Manufacturing Footprint'},
 {'type': 'text', 'text': 'Capital Structure Summary (as of Dec-24)'},
 {'type': 'text', 'text': 'Company Snapshot'},
 {'type': 'text', 'text': 'Business Overview'},
 {'type': 'text', 'text': 'Revenue Split'},
 {'type': 'text', 'text': 'Key Recent Developments'},
 

In [81]:
slides[3]

[{'type': 'text', 'text': 'Organisational Structure'},
 {'type': 'text', 'text': ''},
 {'type': 'text', 'text': 'Organisational Structure'},
 {'type': 'other', 'shape_type': <MSO_SHAPE_TYPE.GROUP: 6>},
 {'type': 'text', 'text': 'Organisational Structure'}]

In [119]:
from pptx.util import Pt

def replace_placeholder_text(file_path: str,placeholder: str,replacement: str,output_path: Optional[str] = None) -> None:
    """
    Replace occurrences of `placeholder` in text elements with `replacement`.

    Modifies the presentation and saves to `output_path` or overwrites original.
    """
    try:
        prs = Presentation(file_path)
    except Exception as e:
        raise IOError(f"Unable to open file {file_path}: {e}")

    for slide in prs.slides:
        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    if placeholder in run.text:
                        run.text = run.text.replace(placeholder, replacement)
                        run.font.size = Pt(5)

    save_path = output_path or file_path
    try:
        prs.save(save_path)
    except Exception as e:
        raise IOError(f"Unable to save updated file to {save_path}: {e}")


In [89]:
file_path = '/Users/felipesilverio/Documents/GitHub/LangChainTest/One-Pager Structure_Reference for Felipe.pptx'

replace_placeholder_text(file_path, 'Company Snapshot', 'Teneo Snapshot','/Users/felipesilverio/Documents/GitHub/LangChainTest/testing.pptx')

# GPT - Sharepoint

In [139]:
my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Bring me back only the code block relative to 1. Company Snapshot, and give me all the information in it in a single line but separated by |

The text is:

{teste}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text('/Users/felipesilverio/Documents/GitHub/LangChainTest/backupppt.pptx', 'Company Snapshot', response['response'], file_path)

my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Bring me back only the code block relative to Business Overview, and give me back exactly as is in the text.

The text is:

{teste}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text(file_path, 'Business Overview Text', response['response'], file_path)

In [140]:
# my_prompt = 'Make the company profile of Tour Partner Group Limited'
# response = chain_with_sources.invoke(my_prompt)

# print(response)
my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Bring me back only the code block relative to 3. Revenue Split, and give me back exactly as is in the text.

The text is:

{teste}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text(file_path, 'Revenue Split', response['response'])

In [141]:
# my_prompt = 'Make the company profile of Tour Partner Group Limited'
# response = chain_with_sources.invoke(my_prompt)

# print(response)
my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Bring me back only the code block relative to Key Stakeholders Table, and give me back exactly as is in the text.

The text is:

{teste}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text(file_path, 'Key Stakeholders Table', response['response'])

In [142]:
# my_prompt = 'Make the company profile of Tour Partner Group Limited'
# response = chain_with_sources.invoke(my_prompt)

# print(response)
my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Find the block Financial Highlits, analyze all metric data and create a table where it is possible to clearly identify the metric and its values.

Return nothing but the table.

The text is:

{teste}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text(file_path, 'Financial Highlights Table', response['response'])

my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Find the block Financial Highlights, and find within it something related to Commentary. 
Analyze all the points that are separated in a numeric list, and create a summary. 

Return nothing but the summary created.

The text is:

{teste}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text(file_path, 'Financial Highlights Commentary', response['response'])

In [143]:
# my_prompt = 'Make the company profile of Tour Partner Group Limited'
# response = chain_with_sources.invoke(my_prompt)

# print(response)
my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Find the block Capital Struct, analyze all metric data and create a table where it is possible to clearly identify the metric and its values.

Return nothing but the table.

The text is:

{teste}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text(file_path, 'Capital Structure Table', response['response'])

my_prompt = f"""
The following text contains a series of text blocks that are separated by multiples - like:
--------------------------------------------------------------------------------------------------------------------

Find the block Capital Struct, and find within it something related to Capital-structure commentary. 
Analyze all the points that are separated in a numeric list, and create a summary. 

Return nothing but the summary created.

The text is:

{teste}
"""
response = chain_with_sources.invoke(my_prompt)
replace_placeholder_text(file_path, 'Capital Structure Commentary', response['response'])

# USING HISTORICAL

In [ ]:
def vector_store_buildup(texts, tables, text_summaries, table_summaries):

    # Validate lengths
    assert len(texts) == len(text_summaries), \
        f"Mismatch: {len(texts)} texts vs {len(text_summaries)} text_summaries"
    assert len(tables) == len(table_summaries), \
        f"Mismatch: {len(tables)} tables vs {len(table_summaries)} table_summaries"

    vectorstore = Chroma(collection_name='multi_modal_rag', embedding_function=OpenAIEmbeddings())

    #Storage layor
    store = InMemoryStore()
    id_key = 'doc_id'

    #retriever

    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )
    #Loading values

    doc_ids = [str(uuid.uuid4()) for _ in texts]
    summary_texts = [
        Document(page_content=summary, metadata={id_key: doc_ids[i]}) for i, summary in enumerate(text_summaries)
    ]
    retriever.vectorstore.add_documents(summary_texts)
    retriever.docstore.mset(list(zip(doc_ids, texts)))

    # Add tables
    tables_ids = [str(uuid.uuid4()) for _ in tables]
    summary_tables = [
        Document(page_content=summary, metadata={id_key: tables_ids[i]}) for i, summary in enumerate(table_summaries)
    ]
    retriever.vectorstore.add_documents(summary_tables)
    retriever.docstore.mset(list(zip(tables_ids, tables)))
    
    return retriever

def parse_docs(docs):
    # Split base64 images and texts
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return{'images':b64, 'texts':text}

def build_prompt(kwargs):
    
    docs_by_type = kwargs['context']
    user_question = kwargs['question']

    context_text = ""
    if len(docs_by_type['texts']) > 0:
        for text_element in docs_by_type['texts']:
            context_text += text_element.text
    else:
        print('No context provided')

    #construct prompt with context
    prompt_template = f""" 
    Answuer the question based only on the following context, which can include text, tables and images.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{'type':'text','text':prompt_template}]

    if len(docs_by_type['images']) > 0:
        for image in docs_by_type['images']:
            prompt_content.append(
                {
                    'type': 'image_url',
                    'image_url': {'url': f'data:image/jpeg;base64,{image}'},
                }
            )

    return ChatPromptTemplate.from_messages([HumanMessage(content=prompt_content),])

def RAG_pipeline(retriever):

    chain = (
            {
                'context': retriever | RunnableLambda(parse_docs),
                'question': RunnablePassthrough(),
            }
            | RunnableLambda(build_prompt)
            | ChatOpenAI(model='gpt-4o-mini')
            | StrOutputParser()
        )

    chain_with_sources = {
        'context': retriever | RunnableLambda(parse_docs),
        'question': RunnablePassthrough(),
        } | RunnablePassthrough().assign(
            response=(
                RunnableLambda(build_prompt)
                | ChatOpenAI(model='gpt-4o')
                | StrOutputParser()
            )
        )

    return chain, chain_with_sources

In [49]:
from dotenv import load_dotenv
from unstructured.partition.pdf import partition_pdf
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from base64 import b64decode
import hashlib
import pickle
import os
from engine2 import EngineJedi
import streamlit as st

def file_md5(path: str) -> str:
    """Compute MD5 hash of a file for caching key."""
    hash_md5 = hashlib.md5()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def file_hash_from_bytes(data: bytes) -> str:
    """Compute MD5 hash of raw PDF bytes for caching key."""
    hash_md5 = hashlib.md5()
    hash_md5.update(data)
    return hash_md5.hexdigest()


def _load_pickle(name: str):
    persist_directory = '/Users/felipesilverio/Documents/GitHub/LangChainTest/chroma_persist/test2.pdf'
    path = os.path.join(persist_directory, f"{name}.pkl")
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def fit():
    # Remembering

    tables_h = _load_pickle('tables')          # list of table elements
    texts_h = _load_pickle('texts')            # list of text elements
    table_summaries_h = _load_pickle('table_summaries')
    text_summaries_h= _load_pickle('text_summaries')

    retriever = vector_store_buildup(texts_h, tables_h, text_summaries_h, table_summaries_h)
    chain, chain_with_sources = RAG_pipeline(retriever)

    return chain, chain_with_sources 
    
chain_h , chain_with_sources_h = fit()

In [50]:
response = chain_with_sources_h.invoke(
    # 'Which companies are mentioned in this file?'
    'What are the two loan facilities: Facility A and Facility B? Provide me a detailed summary'
)

print(response)
print(response['response'])

{'context': {'images': [], 'texts': [<unstructured.documents.elements.Title object at 0x348a58640>]}, 'question': 'What are the two loan facilities: Facility A and Facility B? Provide me a detailed summary', 'response': 'To provide a detailed summary of Term Loan Facilities A & B, refer to the specific details in your provided context. Typically, such facilities differ in terms of their structure, interest rates, maturity dates, repayment schedules, and specific uses. Facility A and Facility B are often part of corporate or structured finance deals aimed at different financing needs, such as capital expenditure, refinancing, or general corporate purposes.\n\nFacility A might be structured with a fixed interest rate and is usually intended for longer-term financing needs. It could have a specific amortization schedule and a defined maturity date.\n\nFacility B could differ by having a floating interest rate tied to a benchmark rate and could be geared towards shorter-term or variable ne